In [1]:
import requests
from bs4 import BeautifulSoup

Gardez à l'esprit que la plupart des sites Web bloquent les requêtes qui ne contiennent pas d'en-tête  User-Agent valide.  
L'en-tête User-Agent de la demande est une chaîne qui caractérise l'application et la version du système d'exploitation d'où provient la demande.

In [3]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

url = 'https://quotes.toscrape.com'
page = requests.get(url, headers=headers)

In [4]:
soup = BeautifulSoup(page.text, 'html.parser')

In [6]:
quotes = []

quote_elements = soup.find_all('div', class_='quote')

for quote_element in quote_elements:

    # extracting the text of the quote
    text = quote_element.find('span', class_='text').text
    
    # extracting the author of the quote
    author = quote_element.find('small', class_='author').text

    # extracting the tag <a> HTML elements related to the quote
    tag_elements = quote_element.find('div', class_='tags').find_all('a', class_='tag')

    # storing the list of tag strings in a list
    tags = []
    for tag_element in tag_elements:
        tags.append(tag_element.text)



Grâce à la find() méthode de Beautiful Soup, vous pouvez extraire le seul élément HTML qui vous intéresse.  
Étant donné que les balises associées au quote sont multiples, vous devez les stocker dans une liste. 

Ensuite, vous pouvez transformer ces données en dictionnaire et les ajouter à la quote liste comme suit :

In [7]:
quotes.append(
    {
        'text': text,
        'author': author,
        'tags': ', '.join(tags) # merging the tags into a "A, B, ..., Z" string
    }
)

In [8]:
print(quotes)

[{'text': '“A day without sunshine is like, you know, night.”', 'author': 'Steve Martin', 'tags': 'humor, obvious, simile'}]


En suivant le lien contenu dans l'élément HTML "Suivant "a", vous pouvez facilement naviguer sur l'ensemble du site.  
Alors, commençons par la page d'accueil et voyons comment parcourir chaque page qui compose le site Web cible.  
Tout ce que vous avez à faire est de rechercher le next "li" élément HTML et d'extraire le lien relatif vers la page suivante.

Vous pouvez implémenter la logique d'exploration comme suit :

In [9]:
# the url of the home page of the target website
base_url = 'https://quotes.toscrape.com'

# retrieving the page and initializing soup...

# getting the "Next →" HTML element
next_li_element = soup.find('li', class_='next')

# if there is a next page to scrape
while next_li_element is not None:
    next_page_relative_url = next_li_element.find('a', href=True)['href']

    # getting the new page
    page = requests.get(base_url + next_page_relative_url, headers=headers)

    # parsing the new page
    soup = BeautifulSoup(page.text, 'html.parser')

    # scraping logic...

    # looking for the "Next →" HTML element in the new page
    next_li_element = soup.find('li', class_='next')


Conversion des données au format CSV

In [10]:
import csv

# scraping logic...

# reading  the "quotes.csv" file and creating it
# if not present
csv_file = open('quotes.csv', 'w', encoding='utf-8', newline='')

# initializing the writer object to insert data
# in the CSV file
writer = csv.writer(csv_file)

# writing the header of the CSV file
writer.writerow(['Text', 'Author', 'Tags'])

# writing each row of the CSV
for quote in quotes:
    writer.writerow(quote.values())

# terminating the operation and releasing the resources
csv_file.close()

Ce que fait cet extrait est d'écrire les données de citation contenues dans la liste des dictionnaires dans un fichier quotes.csv .  
Notez que cela fait partie de la bibliothèque standard Python. Ainsi, vous pouvez l'importer et l'utiliser sans installer de dépendance supplémentaire.  
Dans le détail, il suffit de créer un fichier avec open().  
Ensuite, vous pouvez le remplir avec la fonction writerow() de l'objet Writer de la bibliothèque csv.  
Cela écrira chaque dictionnaire de citation sous la forme d'une ligne au format CSV dans le fichier CSV.

Vous êtes passé de données brutes contenues dans un site Web à des données structurées stockées dans un fichier CSV.  
Le processus d'extraction de données est terminé et vous pouvez maintenant jeter un œil à l'ensemble du scrapp Web Python.

## TOUT ENSEMBLE

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape_page(soup, quotes):
    # retrieving all the quote <div> HTML element on the page
    quote_elements = soup.find_all('div', class_='quote')

    # iterating over the list of quote elements
    # to extract the data of interest and store it
    # in quotes
    for quote_element in quote_elements:
        # extracting the text of the quote
        text = quote_element.find('span', class_='text').text
        # extracting the author of the quote
        author = quote_element.find('small', class_='author').text

        # extracting the tag <a> HTML elements related to the quote
        tag_elements = quote_element.find('div', class_='tags').find_all('a', class_='tag')

        # storing the list of tag strings in a list
        tags = []
        for tag_element in tag_elements:
            tags.append(tag_element.text)

        # appending a dictionary containing the quote data
        # in a new format in the quote list
        quotes.append(
            {
                'text': text,
                'author': author,
                'tags': ', '.join(tags)  # merging the tags into a "A, B, ..., Z" string
            }
        )

# the url of the home page of the target website
base_url = 'https://quotes.toscrape.com'

# defining the User-Agent header to use in the GET request below
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
}

# retrieving the target web page
page = requests.get(base_url, headers=headers)

# parsing the target web page with Beautiful Soup
soup = BeautifulSoup(page.text, 'html.parser')

# initializing the variable that will contain
# the list of all quote data
quotes = []

# scraping the home page
scrape_page(soup, quotes)

# getting the "Next →" HTML element
next_li_element = soup.find('li', class_='next')

# if there is a next page to scrape
while next_li_element is not None:
    next_page_relative_url = next_li_element.find('a', href=True)['href']

    # getting the new page
    page = requests.get(base_url + next_page_relative_url, headers=headers)

    # parsing the new page
    soup = BeautifulSoup(page.text, 'html.parser')

    # scraping the new page
    scrape_page(soup, quotes)

    # looking for the "Next →" HTML element in the new page
    next_li_element = soup.find('li', class_='next')

# reading  the "quotes.csv" file and creating it
# if not present
csv_file = open('quotes.csv', 'w', encoding='utf-8', newline='')

# initializing the writer object to insert data
# in the CSV file
writer = csv.writer(csv_file)

# writing the header of the CSV file
writer.writerow(['Text', 'Author', 'Tags'])

# writing each row of the CSV
for quote in quotes:
    writer.writerow(quote.values())

# terminating the operation and releasing the resources
csv_file.close()